In [88]:
import csv
from datetime import timedelta

import pandas as pd
import pyreadr
from sklearn.linear_model import LinearRegression

In [89]:
substation_df = pd.read_csv('coord_substations.csv')
substation_df

,SUBSTATION,LON,LAT
0,A.COM,2.512180,50.272269
1,HOSTE,-0.625940,44.485958
2,CXSSE,4.511712,48.838806
3,ONDES,1.305962,43.784788


In [90]:
station_df = pd.read_csv('liste_stations.csv', sep=';')
station_df.head()


,IOMM,code_station,Station,NOM_USUEL,Coeff,Code_postal,Code_insee,Reg,Label_reg,Dep,LATITUDE,LONGITUDE,ALTITUDE
0,2,7002,BOULOGNE-SUR-MER,BOULOGNE-SEM,"0,0100",62200,62160,NPP,Hauts-de-France,62,"50,733","1,600",73
1,5,7005,ABBEVILLE,ABBEVILLE,"0,0100",80100,80001,NPP,Hauts-de-France,80,"50,136","1,834",69
2,15,7015,LILLE,LILLE-LESQUIN,"0,0300",59000,59350,NPP,Hauts-de-France,59,"50,570","3,098",47
3,27,7027,CAEN,CAEN-CARPIQUET,"0,0250",14000,14118,NOR,Normandie,14,"49,180","-0,456",67
4,72,7072,REIMS,REIMS-PRUNAY,"0,0000",51100,51454,ACA,Grand-Est,51,"49,210","4,155",95


In [91]:
prod = pyreadr.read_r('prod.RDS')[None]
prod

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC
0,2016-01-01 00:00:00,A.COM,eolien,47.5,104.7,0.453677
1,2016-01-01 00:00:00,CXSSE,eolien,24.2,85.8,0.282051
2,2016-01-01 00:00:00,HOSTE,pv,0.0,54.0,0.000000
3,2016-01-01 00:00:00,national,pv,0.0,6006.0,0.000000
4,2016-01-01 00:00:00,national,eolien,2199.0,10201.0,0.215567
...,...,...,...,...,...,...
420657,2019-12-31 23:00:00,ONDES,pv,NaN,26.8,NaN
420658,2019-12-31 23:30:00,HOSTE,pv,NaN,54.0,NaN
420659,2019-12-31 23:30:00,national,pv,NaN,9258.0,NaN
420660,2019-12-31 23:30:00,national,eolien,NaN,16035.0,NaN


## Météo

In [92]:
meteo_ce_ecmwf = pyreadr.read_r('meteo_ce_allvar_grille_ecmwf.RDS')[None]
meteo_ce_ecmwf.head()

,date_lancement,zone,date_cible,u100,v100,ff100,neb,t2m,ssrd,echeance
0,2016-01-01 12:00:00,SEO,2016-01-01 12:00:00,-5.177443,6.064677,8.121044,0.939656,8.877014,185.685528,0.0
1,2016-01-01 12:00:00,SEO,2016-01-01 12:30:00,-5.469025,6.075288,8.323454,0.886481,9.188167,194.134987,0.5
2,2016-01-01 12:00:00,SEO,2016-01-01 13:00:00,-5.827872,6.013414,8.541550,0.831598,9.479097,200.354708,1.0
3,2016-01-01 12:00:00,SEO,2016-01-01 13:30:00,-6.272080,5.900551,8.814541,0.780358,9.727099,193.196003,1.5
4,2016-01-01 12:00:00,SEO,2016-01-01 14:00:00,-6.623078,6.134103,9.275718,0.766335,9.899554,163.905746,2.0


In [93]:
meteo_nationale_ecmwf = meteo_ce_ecmwf[meteo_ce_ecmwf.zone=="SEO"] #TODO FIXME !!!
meteo_nationale_ecmwf.rename(columns={'zone':'perimetre_prod'},inplace=True)
meteo_nationale_ecmwf.perimetre_prod = 'national'
meteo_nationale_ecmwf.head()

/home/rozierale/.miniconda3/envs/origami/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/rozierale/.miniconda3/envs/origami/lib/python3.7/site-packages/pandas/core/generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,date_lancement,perimetre_prod,date_cible,u100,v100,ff100,neb,t2m,ssrd,echeance
0,2016-01-01 12:00:00,national,2016-01-01 12:00:00,-5.177443,6.064677,8.121044,0.939656,8.877014,185.685528,0.0
1,2016-01-01 12:00:00,national,2016-01-01 12:30:00,-5.469025,6.075288,8.323454,0.886481,9.188167,194.134987,0.5
2,2016-01-01 12:00:00,national,2016-01-01 13:00:00,-5.827872,6.013414,8.541550,0.831598,9.479097,200.354708,1.0
3,2016-01-01 12:00:00,national,2016-01-01 13:30:00,-6.272080,5.900551,8.814541,0.780358,9.727099,193.196003,1.5
4,2016-01-01 12:00:00,national,2016-01-01 14:00:00,-6.623078,6.134103,9.275718,0.766335,9.899554,163.905746,2.0


In [94]:
meteo_ce_nebtemp_mf = pyreadr.read_r('meteo_ce_nebtemp_station_meteofrance.RDS')[None]
meteo_ce_nebtemp_mf.head()

,date_lancement,zone,date_cible,t2m,neb,echeance
0,2015-12-29 12:00:00,France,2016-01-01 00:00:00,5.885630,4.426500,60.0
1,2015-12-29 12:00:00,France,2016-01-01 00:30:00,5.760679,4.400543,60.5
2,2015-12-29 12:00:00,France,2016-01-01 01:00:00,5.646430,4.404500,61.0
3,2015-12-29 12:00:00,France,2016-01-01 01:30:00,5.542821,4.404271,61.5
4,2015-12-29 12:00:00,France,2016-01-01 02:00:00,5.443720,4.398000,62.0


In [95]:
meteo_ce_vent_mf = pyreadr.read_r('meteo_ce_vent_grille_meteofrance.RDS')[None]
meteo_ce_vent_mf.head()

,date_lancement,zone,date_cible,u100,v100,ff100,echeance
0,2015-12-29 12:00:00,France,2016-01-01,2.325891,5.899836,7.304124,60.0
1,2015-12-29 12:00:00,SEE,2016-01-01,3.143937,4.076005,5.298578,60.0
2,2015-12-29 12:00:00,SENE,2016-01-01,2.406227,8.448678,8.849158,60.0
3,2015-12-29 12:00:00,SENP,2016-01-01,3.430701,10.169736,10.791216,60.0
4,2015-12-29 12:00:00,SEO,2016-01-01,2.398538,7.153337,7.911717,60.0


In [96]:
meteo_locale_ecmwf = pyreadr.read_r('meteo_locale_allvar_grille_ecmwf.RDS')[None]
meteo_locale_ecmwf.head()

,date_lancement,perimetre_prod,date_cible,tcc,t2m,ssrd,u100,v100,ff100,echeance
0,2016-01-01 12:00:00,A.COM,2016-01-01 12:00:00,1.000000,5.433659,74.616463,-5.155345,4.644291,6.943392,0.0
1,2016-01-01 12:00:00,A.COM,2016-01-01 12:30:00,1.000000,5.761281,110.394319,-5.664127,5.249110,7.722563,0.5
2,2016-01-01 12:00:00,A.COM,2016-01-01 13:00:00,1.000000,6.026459,84.842507,-5.938574,5.744662,8.262706,1.0
3,2016-01-01 12:00:00,A.COM,2016-01-01 13:30:00,1.000001,6.188425,40.860147,-5.851014,6.026186,8.401581,1.5
4,2016-01-01 12:00:00,A.COM,2016-01-01 14:00:00,1.000001,6.293115,23.576228,-5.700430,6.006944,8.284104,2.0


In [97]:
meteo_locale_vent_mf = pyreadr.read_r('meteo_locale_vent_grille_meteofrance.RDS')[None]
meteo_locale_vent_mf.head()

,date_lancement,perimetre_prod,date_cible,u100,v100,ff100,echeance
0,2015-12-29 12:00:00,A.COM,2016-01-01 00:00:00,3.101111,10.747778,11.191169,60.0
1,2015-12-30 12:00:00,A.COM,2016-01-01 00:00:00,5.330000,10.257778,11.562233,36.0
2,2015-12-30 12:00:00,A.COM,2016-01-01 00:30:00,5.793723,9.453217,11.143628,36.5
3,2015-12-30 12:00:00,A.COM,2016-01-01 01:00:00,6.132471,8.487282,10.590947,37.0
4,2015-12-30 12:00:00,A.COM,2016-01-01 01:30:00,6.352303,7.430982,9.940086,37.5


## Jointure prod + meteo

In [98]:
def keep_valid_forecasts(row):
    veille = (row.date_cible - timedelta(days=1))
    earliest_valid_lancement = veille.replace(hour=16)
    return row.date_lancement < earliest_valid_lancement
meteo_df = pd.concat([meteo_nationale_ecmwf, meteo_locale_ecmwf],axis=0)
train_meteo_df = meteo_df[meteo_df.apply(keep_valid_forecasts, axis=1)]
train_meteo_df

,date_lancement,perimetre_prod,date_cible,u100,v100,ff100,neb,t2m,ssrd,echeance,tcc
24,2016-01-01 12:00:00,national,2016-01-02 00:00:00,5.090201,11.107833,12.586564,0.738982,10.223554,7.579537e-05,12.0,NaN
25,2016-01-01 12:00:00,national,2016-01-02 00:30:00,5.744563,11.009990,12.766400,0.711113,10.179822,1.905824e-19,12.5,NaN
26,2016-01-01 12:00:00,national,2016-01-02 01:00:00,6.332025,10.756720,12.839232,0.670307,10.132119,-6.648113e-04,13.0,NaN
27,2016-01-01 12:00:00,national,2016-01-02 01:30:00,6.799050,10.373602,12.779465,0.616743,10.072019,1.905824e-19,13.5,NaN
28,2016-01-01 12:00:00,national,2016-01-02 02:00:00,7.192248,9.944114,12.658765,0.565732,9.992211,2.583450e-03,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
846403,2019-12-30 12:00:00,ONDES,2019-12-31 21:30:00,2.682430,-0.612226,2.774608,NaN,5.987815,0.000000e+00,33.5,0.589047
846404,2019-12-30 12:00:00,ONDES,2019-12-31 22:00:00,2.946498,-0.379447,3.007768,NaN,5.711979,8.224577e-03,34.0,0.623447
846405,2019-12-30 12:00:00,ONDES,2019-12-31 22:30:00,3.167307,-0.073829,3.218369,NaN,5.737721,0.000000e+00,34.5,0.679307
846406,2019-12-30 12:00:00,ONDES,2019-12-31 23:00:00,3.357581,0.190312,3.400280,NaN,5.867768,-2.223821e-03,35.0,0.730482


In [99]:
# Keep only latest date_lancement
train_meteo_df = train_meteo_df.groupby(['perimetre_prod', 'date_cible']).last().reset_index()
train_meteo_df

,perimetre_prod,date_cible,date_lancement,u100,v100,ff100,neb,t2m,ssrd,echeance,tcc
0,A.COM,2016-01-02 00:00:00,2016-01-01 12:00:00,-5.435037,9.539942,10.980536,NaN,6.325450,2.216240e-03,12.0,1.000000
1,A.COM,2016-01-02 00:30:00,2016-01-01 12:00:00,-5.078984,9.433556,10.707797,NaN,6.344030,0.000000e+00,12.5,0.999551
2,A.COM,2016-01-02 01:00:00,2016-01-01 12:00:00,-4.810986,9.301626,10.473833,NaN,6.317258,-1.425500e-03,13.0,1.000000
3,A.COM,2016-01-02 01:30:00,2016-01-01 12:00:00,-4.448654,9.288761,10.308406,NaN,6.246803,-7.623297e-19,13.5,1.000689
4,A.COM,2016-01-02 02:00:00,2016-01-01 12:00:00,-4.109660,9.396701,10.258342,NaN,6.219141,3.485759e-03,14.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
350395,national,2019-12-31 21:30:00,2019-12-30 12:00:00,-0.350092,4.108127,4.503309,0.821373,6.304769,0.000000e+00,33.5,NaN
350396,national,2019-12-31 22:00:00,2019-12-30 12:00:00,-0.377017,4.076441,4.620322,0.805843,6.389925,3.180431e-03,34.0,NaN
350397,national,2019-12-31 22:30:00,2019-12-30 12:00:00,-0.420894,4.051164,4.868887,0.797661,6.404927,3.049319e-18,34.5,NaN
350398,national,2019-12-31 23:00:00,2019-12-30 12:00:00,-0.442239,4.061788,5.069467,0.802324,6.404885,-8.798645e-04,35.0,NaN


In [100]:
# Check if only one date_lancement per date_cible
train_meteo_df.groupby(['perimetre_prod', 'date_cible']).count()

date_lancement  u100  v100  ff100  neb  \
perimetre_prod date_cible                                                    
A.COM          2016-01-02 00:00:00               1     1     1      1    0   
               2016-01-02 00:30:00               1     1     1      1    0   
               2016-01-02 01:00:00               1     1     1      1    0   
               2016-01-02 01:30:00               1     1     1      1    0   
               2016-01-02 02:00:00               1     1     1      1    0   
...                                            ...   ...   ...    ...  ...   
national       2019-12-31 21:30:00               1     1     1      1    1   
               2019-12-31 22:00:00               1     1     1      1    1   
               2019-12-31 22:30:00               1     1     1      1    1   
               2019-12-31 23:00:00               1     1     1      1    1   
               2019-12-31 23:30:00               1     1     1      1    1   

                                    t2m  ssrd  echeance  tcc  
perimetre_prod date_cible                                     
A.COM          2016-01-02 00:00:00    1     1         1    1  
               2016-01-02 00:30:00    1     1         1    1  
               2016-01-02 01:00:00    1     1         1    1  
               2016-01-02 01:30:00    1     1         1    1  
               2016-01-02 02:00:00    1     1         1    1  
...                                 ...   ...       ...  ...  
national       2019-12-31 21:30:00    1     1         1    0  
               2019-12-31 22:00:00    1     1         1    0  
               2019-12-31 22:30:00    1     1         1    0  
               2019-12-31 23:00:00    1     1         1    0  
               2019-12-31 23:30:00    1     1         1    0  

[350400 rows x 9 columns]

In [101]:
train_prod = prod[prod.date_cible.dt.year < 2019]
test_prod = prod[prod.date_cible.dt.year == 2019]

train_prod

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC
0,2016-01-01 00:00:00,A.COM,eolien,47.500000,104.7,0.453677
1,2016-01-01 00:00:00,CXSSE,eolien,24.200000,85.8,0.282051
2,2016-01-01 00:00:00,HOSTE,pv,0.000000,54.0,0.000000
3,2016-01-01 00:00:00,national,pv,0.000000,6006.0,0.000000
4,2016-01-01 00:00:00,national,eolien,2199.000000,10201.0,0.215567
...,...,...,...,...,...,...
315631,2018-12-31 23:30:00,CXSSE,eolien,0.166667,85.8,0.001943
315632,2018-12-31 23:30:00,HOSTE,pv,0.000000,54.0,0.000000
315633,2018-12-31 23:30:00,national,pv,0.000000,8328.0,0.000000
315634,2018-12-31 23:30:00,national,eolien,1612.000000,14500.0,0.111172


In [102]:
train_df = train_prod.merge(train_meteo_df, on=['perimetre_prod','date_cible'], how='inner')
test_df = test_prod.merge(train_meteo_df, on=['perimetre_prod','date_cible'], how='inner')
train_df

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC,date_lancement,u100,v100,ff100,neb,t2m,ssrd,echeance,tcc
0,2016-01-02 00:00:00,A.COM,eolien,63.033333,104.7,0.602038,2016-01-01 12:00:00,-5.435037,9.539942,10.980536,NaN,6.325450,2.216240e-03,12.0,1.000000
1,2016-01-02 00:00:00,CXSSE,eolien,57.766667,85.8,0.673271,2016-01-01 12:00:00,-4.723014,6.589204,8.123462,NaN,6.039697,7.531540e-04,12.0,0.999763
2,2016-01-02 00:00:00,HOSTE,pv,0.000000,54.0,0.000000,2016-01-01 12:00:00,4.830697,2.293848,5.401187,NaN,9.673486,-2.713449e-03,12.0,0.999146
3,2016-01-02 00:00:00,national,pv,0.000000,6006.0,0.000000,2016-01-01 12:00:00,5.090201,11.107833,12.586564,0.738982,10.223554,7.579537e-05,12.0,NaN
4,2016-01-02 00:00:00,national,eolien,5543.000000,10201.0,0.543378,2016-01-01 12:00:00,5.090201,11.107833,12.586564,0.738982,10.223554,7.579537e-05,12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315343,2018-12-31 23:30:00,CXSSE,eolien,0.166667,85.8,0.001943,2018-12-30 12:00:00,1.297931,-3.794680,4.078872,NaN,5.548341,0.000000e+00,35.5,0.784705
315344,2018-12-31 23:30:00,HOSTE,pv,0.000000,54.0,0.000000,2018-12-30 12:00:00,-3.254897,-3.460821,4.753223,NaN,3.432281,-1.524659e-18,35.5,0.184643
315345,2018-12-31 23:30:00,national,pv,0.000000,8328.0,0.000000,2018-12-30 12:00:00,-1.700964,-3.081915,3.918188,0.866119,6.909247,-7.623297e-19,35.5,NaN
315346,2018-12-31 23:30:00,national,eolien,1612.000000,14500.0,0.111172,2018-12-30 12:00:00,-1.700964,-3.081915,3.918188,0.866119,6.909247,-7.623297e-19,35.5,NaN


### Build 6 datasets

In [103]:
features = ['puissance_installee','u100','v100','ff100','neb','t2m','ssrd','tcc']

train_df

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC,date_lancement,u100,v100,ff100,neb,t2m,ssrd,echeance,tcc
0,2016-01-02 00:00:00,A.COM,eolien,63.033333,104.7,0.602038,2016-01-01 12:00:00,-5.435037,9.539942,10.980536,NaN,6.325450,2.216240e-03,12.0,1.000000
1,2016-01-02 00:00:00,CXSSE,eolien,57.766667,85.8,0.673271,2016-01-01 12:00:00,-4.723014,6.589204,8.123462,NaN,6.039697,7.531540e-04,12.0,0.999763
2,2016-01-02 00:00:00,HOSTE,pv,0.000000,54.0,0.000000,2016-01-01 12:00:00,4.830697,2.293848,5.401187,NaN,9.673486,-2.713449e-03,12.0,0.999146
3,2016-01-02 00:00:00,national,pv,0.000000,6006.0,0.000000,2016-01-01 12:00:00,5.090201,11.107833,12.586564,0.738982,10.223554,7.579537e-05,12.0,NaN
4,2016-01-02 00:00:00,national,eolien,5543.000000,10201.0,0.543378,2016-01-01 12:00:00,5.090201,11.107833,12.586564,0.738982,10.223554,7.579537e-05,12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315343,2018-12-31 23:30:00,CXSSE,eolien,0.166667,85.8,0.001943,2018-12-30 12:00:00,1.297931,-3.794680,4.078872,NaN,5.548341,0.000000e+00,35.5,0.784705
315344,2018-12-31 23:30:00,HOSTE,pv,0.000000,54.0,0.000000,2018-12-30 12:00:00,-3.254897,-3.460821,4.753223,NaN,3.432281,-1.524659e-18,35.5,0.184643
315345,2018-12-31 23:30:00,national,pv,0.000000,8328.0,0.000000,2018-12-30 12:00:00,-1.700964,-3.081915,3.918188,0.866119,6.909247,-7.623297e-19,35.5,NaN
315346,2018-12-31 23:30:00,national,eolien,1612.000000,14500.0,0.111172,2018-12-30 12:00:00,-1.700964,-3.081915,3.918188,0.866119,6.909247,-7.623297e-19,35.5,NaN


In [104]:

regressor = LinearRegression()

In [105]:
regressor.fit(train_df[features].dropna(axis='columns'),train_df['FC'])

LinearRegression()

In [106]:
test_df['prev_FC'] = regressor.predict(test_df[features].dropna(axis='columns'))

In [108]:
test_df_result = test_df.copy()
def is_target_of_challenge(row):
    if row.perimetre_prod=='A.COM' and row.type_prod =='eolien':
        return True
    elif row.perimetre_prod=='CXSSE' and row.type_prod =='eolien':
        return True
    elif row.perimetre_prod=='HOSTE' and row.type_prod =='pv':
        return True
    elif row.perimetre_prod=='ONDES' and row.type_prod =='pv':
        return True
    elif row.perimetre_prod=='national' and row.type_prod in ['pv','eolien']:
        return True
#test_df_result.date_cible = test_df_result.date_cible.apply(str)
test_df_result = test_df_result[test_df_result.apply(is_target_of_challenge,axis=1)]

In [111]:
test_df_result[['date_cible','perimetre_prod','type_prod','prev_FC']]\
    .to_csv('preds_alexandre_rozier.csv',sep=';',index=False,date_format='%Y-%m-%dT%H:%M:%SZ',float_format="%.3f",decimal='.')

